In [1]:
import access

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= access.c_id,
                                                           client_secret= access.c_se))

## Loading Dataframe

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from csv import DictReader
import random
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
df_songs = pd.read_csv("spotify_2_clustering.csv")
df_songs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Clusters
0,0.220,0.00528,1,-36.410,0,0.0457,0.99500,0.94400,0.1290,0.3040,74.434,362787,4,0
1,0.210,0.14100,0,-15.641,1,0.0363,0.35300,0.97300,0.2440,0.0381,98.395,119467,3,6
2,0.210,0.14100,0,-15.641,1,0.0363,0.35300,0.97300,0.2440,0.0381,98.395,119467,3,6
3,0.477,0.00612,5,-36.199,1,0.0587,0.99600,0.94300,0.0702,0.3560,101.196,161600,3,0
4,0.388,0.23100,4,-15.395,0,0.0321,0.92100,0.96800,0.1180,0.5330,139.654,125733,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,0.659,0.62800,0,-8.685,0,0.1790,0.18600,0.00000,0.1050,0.2620,94.050,219427,4,5
2478,0.388,0.63100,4,-6.328,0,0.0391,0.00939,0.00474,0.3720,0.2760,112.033,201432,3,2
2479,0.538,0.55700,11,-8.678,1,0.0914,0.26200,0.91400,0.1020,0.5060,131.030,231724,4,9
2480,0.811,0.72600,0,-3.995,1,0.0813,0.15400,0.00000,0.1160,0.6660,131.027,264920,4,8


## Input Song

In [5]:
songname = input("Enter the song and name of its artist: ")
songname = songname.lower()

results = sp.search(q = songname,limit=5, market="DE")

Enter the song and name of its artist: attrapen


In [6]:
l_artist = []
ids = []
for index in range (0, len(results["tracks"]["items"])):
    l_artist.append(results["tracks"]["items"][index]['artists'][0]["name"])
    ids.append(results["tracks"]["items"][index]['id'])
ids

['6LuAtG9sa0aU8GTMd9CQnO',
 '1dlAbh3GqgntOBvW6CLQRT',
 '6ro7bmnTuOhGoKeYFRPN3G',
 '49lh0GYyryaMVGkW7J0ZfG',
 '0aTaTaROrLwQokUSudVKIe']

In [7]:
l_artist


['Sofiane', 'XATAR', 'Polo & Pan', 'Hatik', 'Polo & Pan']

In [8]:
print(l_artist)
choose_artist = input("Choose your artist: 1, 2, 3, 4 , 5?")

['Sofiane', 'XATAR', 'Polo & Pan', 'Hatik', 'Polo & Pan']
Choose your artist: 1, 2, 3, 4 , 5?2


In [9]:
track_id = results["tracks"]["items"][int(choose_artist)-1]['id']
track_id

'1dlAbh3GqgntOBvW6CLQRT'

In [ ]:
df_songs = df_songs["track_id"]

In [10]:
from IPython.display import IFrame

#track_id = "0vHtlgsJoyA91YMNdImnCL"

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

## Getting Audio Features

In [11]:
list_af = []

for item in results:
    sp.audio_features(track_id)
    list_af.append(sp.audio_features(track_id))
    print(sp.audio_features(track_id))

[{'danceability': 0.585, 'energy': 0.765, 'key': 8, 'loudness': -6.542, 'mode': 1, 'speechiness': 0.389, 'acousticness': 0.175, 'instrumentalness': 0, 'liveness': 0.0846, 'valence': 0.703, 'tempo': 90.823, 'type': 'audio_features', 'id': '1dlAbh3GqgntOBvW6CLQRT', 'uri': 'spotify:track:1dlAbh3GqgntOBvW6CLQRT', 'track_href': 'https://api.spotify.com/v1/tracks/1dlAbh3GqgntOBvW6CLQRT', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1dlAbh3GqgntOBvW6CLQRT', 'duration_ms': 232080, 'time_signature': 4}]


In [12]:
#list_af = [item[0] for item in list_af]

In [13]:
df_af = pd.DataFrame(list_af[0])

In [14]:
df_af

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.585,0.765,8,-6.542,1,0.389,0.175,0,0.0846,0.703,90.823,audio_features,1dlAbh3GqgntOBvW6CLQRT,spotify:track:1dlAbh3GqgntOBvW6CLQRT,https://api.spotify.com/v1/tracks/1dlAbh3Gqgnt...,https://api.spotify.com/v1/audio-analysis/1dlA...,232080,4


In [15]:
df_af = df_af.drop(["type","uri","track_href","analysis_url","id"], axis = 1)

In [16]:
X =df_af
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.585,0.765,8,-6.542,1,0.389,0.175,0,0.0846,0.703,90.823,232080,4


## Loading the StandardScaler and import pickle

In [17]:
def load_pickle(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 


In [18]:
scaler = load_pickle(r"C:\Users\Mauricio\Documents\GitHub\da_mauricio\Spotify\scaler.pickle")
model_kmeans = load_pickle(r"C:\Users\Mauricio\Documents\GitHub\da_mauricio\Spotify\kmeans_11.pickle")

In [19]:
columns = X.columns
columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature'],
      dtype='object')

In [20]:
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

In [21]:
X_scaled_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,-0.267916,0.419513,0.742332,0.293702,0.886347,3.682706,-0.268218,-0.502459,-0.66603,0.570857,-1.125706,-0.034714,0.183536


In [22]:
labels = model_kmeans.predict(X_scaled_df)

In [23]:
labels

array([10])

In [24]:
clusters = kmeans.predict(X_scaled_df)
#clusters
#pd.Series(clusters).value_counts().sort_index()
#kmeans.inertia_
clusters

NameError: name 'kmeans' is not defined

In [ ]:
recommendation =X_scaled_df[X_scaled_df["cluster"] ==labels[0]].sample()